In [1]:
import os
import torch

from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json

import numpy as np
import pandas as pd
import seaborn as sns

In [9]:
pred_df = pd.read_csv('/home/prml/StudentsWork/ChanYoung/Experiments/rt_detr/AAAI2025/superclass_silu2333333/pred_iou.csv')

# pred_df.drop(columns=['pred_box', 'gt_box'], inplace=True)


pred_df['gt_ann_id'].fillna(-1, inplace=True)

pred_df['gt_ann_id'] = pred_df['gt_ann_id'].astype(int)
pred_df['gt_class'] = pred_df['gt_class'].astype(int)
pred_df.head()

/tmp/ipykernel_1600828/746830903.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pred_df['gt_ann_id'].fillna(-1, inplace=True)


,pred_score,pred_box,max_iou,pred_class,gt_class,gt_box,img_id,gt_ann_id
0,0.942287,"[384.7174377441406, 68.72052001953125, 499.659...",0.946754,1,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887
1,0.918423,"[57.57688522338867, 288.3216552734375, 134.855...",0.925807,7,51,"[59.630001068115234, 287.3599853515625, 135.69...",397133,716434
2,0.897394,"[30.68769645690918, 343.5014343261719, 99.1896...",0.961529,7,51,"[31.280000686645508, 344.0, 99.4000015258789, ...",397133,713388
3,0.831097,"[154.79803466796875, 167.99612426757812, 183.4...",0.825147,7,51,"[155.97000122070312, 168.9499969482422, 182.0,...",397133,1902250
4,0.824789,"[0.033206939697265625, 263.4237060546875, 60.3...",0.932011,1,1,"[0.0, 262.80999755859375, 62.15999984741211, 2...",397133,1218137


In [10]:
pred_df[pred_df['gt_ann_id'] == 200887]

,pred_score,pred_box,max_iou,pred_class,gt_class,gt_box,img_id,gt_ann_id
0,0.942287,"[384.7174377441406, 68.72052001953125, 499.659...",0.946754,1,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887
19,0.507490,"[489.107666015625, 44.52312088012695, 498.9218...",0.005863,7,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887
28,0.428483,"[380.2354736328125, 46.07578659057617, 392.392...",0.003024,7,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887
29,0.421881,"[477.55059814453125, 40.877044677734375, 485.6...",0.006309,7,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887
45,0.348473,"[477.591552734375, 46.53861618041992, 484.8574...",0.005643,7,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887
68,0.257907,"[452.4635314941406, 37.27126693725586, 465.985...",0.001303,7,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887
70,0.252856,"[476.34344482421875, 43.00468826293945, 485.19...",0.006922,7,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887
95,0.223745,"[487.9461975097656, 44.467041015625, 511.59945...",0.014562,7,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887
96,0.223106,"[397.74749755859375, 0.5962703227996826, 416.1...",0.006883,7,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887
98,0.222924,"[429.88226318359375, 114.47142791748047, 469.1...",0.082784,5,1,"[388.6600036621094, 69.91999816894531, 498.070...",397133,200887


In [3]:
real_df = pd.read_csv('/home/prml/StudentsWork/ChanYoung/Experiments/rt_detr/AAAI2025/superclass_silu2333333//real_iou.csv')

real_df.drop(columns=['pred_box', 'gt_box'], inplace=True)

real_df.head()

,pred_score,max_iou,pred_class,gt_class,gt_ann_id,gt_img_id
0,0.206647,0.869916,7,44,82445,397133
1,0.562286,0.940867,9,67,119568,397133
2,0.942287,0.946754,1,1,200887,397133
3,0.532361,0.978436,7,49,693231,397133
4,0.241727,0.977982,7,51,713388,397133


-------------------------------

In [4]:
pred_gt_anns = pred_df[(pred_df['max_iou'] >= 0.5) & (pred_df['pred_score'] >= 0.5)]['gt_ann_id'].unique()
len(pred_gt_anns)

25502

In [5]:
check_pred_gt_anns = pred_df[(pred_df['max_iou'] >= 0.5) & (pred_df['pred_score'] >= 0.0)]['gt_ann_id'].unique()
len(check_pred_gt_anns)

34138

In [6]:
real_gt_anns = real_df['gt_ann_id'].unique()
len(real_gt_anns)

36781

In [7]:
# recall은
len(pred_gt_anns) / len(real_gt_anns)

0.6933471085614855

In [8]:
pred_gt_anns = set(pred_df[(pred_df['max_iou'] >= 0.75) & (pred_df['pred_score'] >= 0.75)]['gt_ann_id'].unique())
miss_gt_anns = set(real_df[(real_df['max_iou'] >= 0.75) & (real_df['pred_score'] <= 0.75)]['gt_ann_id'].unique())

print(len(miss_gt_anns-pred_gt_anns) / len(miss_gt_anns))


0.7037699832975424


----------------------------------

In [16]:
miss_ann_ids = []
for ann_id in real_df[(real_df['pred_score'] < 0.5) & (real_df['max_iou'] > 0.5)]['gt_ann_id'].unique():
    check = pred_df[pred_df['gt_ann_id'] == ann_id]
    
    if check.empty or check['pred_score'].max() < 0.5:
        miss_ann_ids.append(ann_id)
    
len(miss_ann_ids)

6755

In [17]:
len(miss_ann_ids)/len(real_df['gt_ann_id'].unique())

0.18365460427938338